In [1]:
import os
import pytz
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps
from flourish_caregiver.models.signals import cohort_assigned
from flourish_caregiver.helper_classes import SequentialCohortEnrollment
from flourish_caregiver.helper_classes.schedule_dict import child_schedule_dict, caregiver_schedule_dict
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment
from edc_appointment.constants import NEW_APPT
from edc_visit_schedule import site_visit_schedules
from edc_base.utils import get_utcnow, age
from tqdm import tqdm

In [2]:
child_identifiers = CaregiverChildConsent.objects.exclude(
    preg_enroll=True).values_list('subject_identifier', flat=True).distinct()
child_identifiers = list(set(child_identifiers))[1:]

In [3]:
# Create cohort instance for enrolment cohort
count = 0
for idx in tqdm(child_identifiers):
    childconsent_obj = CaregiverChildConsent.objects.filter(subject_identifier=idx).earliest('consent_datetime')
    obj, created = Cohort.objects.get_or_create(
        subject_identifier=childconsent_obj.subject_identifier,
        name=childconsent_obj.cohort,
        defaults={'assign_datetime': get_utcnow(), 'enrollment_cohort': True})
    if created:
        count += 1
print(f'Created {count} cohort instance(s)')

100%|██████████| 989/989 [00:43<00:00, 22.87it/s]

Created 0 cohort instance(s)


In [4]:
# Current age cohort recalculation
records = []
for idx in tqdm(child_identifiers):
    childconsent_obj = CaregiverChildConsent.objects.filter(subject_identifier=idx).earliest('consent_datetime')
    if not childconsent_obj.is_preg:
        enrol_cohort = childconsent_obj.cohort
        cohort = cohort_assigned(
            childconsent_obj.study_child_identifier,
            childconsent_obj.child_dob,
            get_utcnow().date())

        child_age = age(childconsent_obj.child_dob, get_utcnow().date())
        child_age = round(child_age.years + (child_age.months/12) + (child_age.days/365.25), 2)
        caregiver_subject_identifier = childconsent_obj.subject_consent.subject_identifier
        if (cohort and enrol_cohort != cohort and cohort != f'{enrol_cohort}_sec'):
            records.append({'subject_identifier': idx,
                            'caregiver_subject_identifier': caregiver_subject_identifier,
                            'child_count': childconsent_obj.caregiver_visit_count,
                            'enrolment_cohort': enrol_cohort,
                            'current_age': child_age,
                            'current_cohort': cohort, })
#         else:
#             record.update({'current_cohort': enrol_cohort, })

100%|██████████| 989/989 [02:31<00:00,  6.53it/s]


In [5]:
def take_off_schedule(onschedule_model_obj):
    subject_identifier = onschedule_model_obj.subject_identifier
    schedule_name = onschedule_model_obj.schedule_name
    onschedule_model = onschedule_model_obj.onschedule_model

    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=onschedule_model, name=schedule_name)

    if schedule.is_onschedule(
        subject_identifier=subject_identifier, report_datetime=get_utcnow()):
        schedule.take_off_schedule(
            subject_identifier=subject_identifier,
            schedule_name=schedule_name)


In [6]:
def delete_completed_appointments(appointment_model_cls, subject_identifier,
                                  schedule_name ):
    complete_appts = appointment_model_cls.objects.filter(
        Q(schedule_name__icontains='quart') | Q(schedule_name__icontains='qt'),
        subject_identifier=subject_identifier, ).exclude(
            appt_status=NEW_APPT).values_list('visit_code', flat=True).distinct()

    new_appts = appointment_model_cls.objects.filter(
        subject_identifier=subject_identifier,
        schedule_name=schedule_name,
        visit_code__in=complete_appts)
    if new_appts.exists():
        new_appts.delete()

In [7]:
def update_onschedule_model(subject_identifier, child_subject_identifier,
                            onschedule_model, schedule_name, ):

    onschedule_model_cls = django_apps.get_model(onschedule_model)
    try:
        onschedule_model_cls.objects.get(
            subject_identifier=subject_identifier,
            schedule_name=schedule_name,
            child_subject_identifier=child_subject_identifier)
    except onschedule_model_cls.DoesNotExist:
        try:
            onschedule_obj = onschedule_model_cls.objects.get(
                subject_identifier=subject_identifier,
                schedule_name=schedule_name)
        except schedule.onschedule_model_cls.DoesNotExist:
            pass
        else:
            onschedule_obj.child_subject_identifier = child_subject_identifier
            onschedule_obj.save()

In [8]:
def put_on_schedule(onschedule_model, schedule_name,
                    subject_identifier, base_appt_datetime=None, ):
    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=onschedule_model,
        name=schedule_name)

    schedule.put_on_schedule(
        subject_identifier=subject_identifier,
        onschedule_datetime=get_utcnow(),
        base_appt_datetime=base_appt_datetime,
        schedule_name=schedule_name)

In [9]:
def put_onschedule(cohort, schedule_type, schedule_dict, onschedule_datetime,
                   subject_identifier, appointment_cls, is_caregiver=False, child_count=None):
    onschedule_model = schedule_dict[cohort][schedule_type]['onschedule_model']
    if is_caregiver:
        schedule_name=caregiver_schedule_dict[cohort][schedule_type][child_count]
    else:
        schedule_name = schedule_dict[cohort][schedule_type]['name']

    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
            onschedule_model=onschedule_model,
            name=schedule_name)

    if not schedule.is_onschedule(
        subject_identifier=subject_identifier, report_datetime=onschedule_datetime):

        put_on_schedule(onschedule_model=onschedule_model,
                        schedule_name=schedule_name,
                        base_appt_datetime=onschedule_datetime,
                        subject_identifier=subject_identifier)

        delete_completed_appointments(
                appointment_model_cls=appointment_cls,
                subject_identifier=subject_identifier,
                schedule_name=schedule_name)
    return onschedule_model, schedule_name
        

In [10]:
def get_onschedule_obj(subject_identifier, cohort):
    cohort = f'{cohort}_' if 'sec' not in cohort else cohort
    try:
        schedule_obj = SubjectScheduleHistory.objects.filter(
            subject_identifier=subject_identifier,
            schedule_name__icontains=cohort.replace('cohort_', '')).filter(
            Q(schedule_name__icontains='qt') | Q(schedule_name__icontains='quart')).latest('onschedule_datetime')
    except SubjectScheduleHistory.DoesNotExist:
        raise
    else:
        return schedule_obj

In [11]:
def get_schedule_type(onschedule_obj):
    schedule_name = onschedule_obj.schedule_name
    if 'fu' in schedule_name:
        return 'followup_quarterly'
    else:
        return 'quarterly'

In [12]:
"""
    Take participant offschedule for enrolment cohort and put onschedule for subsequent cohort schedule
    1. Create a cohort instance for `current cohort`,
        NOTE: If cohort instance already exists assume already put onschedule
    2. Take offschedule for `enrolment cohort`
    3. Put onschedule for `current cohort`
    4. Make sure to align appointments for `current cohort` schedule with enrolment
"""
for cohort_change in tqdm(records):
    enrol_cohort = cohort_change.get('enrolment_cohort')
    current_cohort = cohort_change.get('current_cohort')
    subject_identifier=cohort_change.get('subject_identifier')
    caregiver_subject_identifier=cohort_change.get('caregiver_subject_identifier')
    if subject_identifier == 'B142-040990341-5-10':
        continue

    print('start', subject_identifier)
    obj, created = Cohort.objects.get_or_create(
        subject_identifier=subject_identifier,
        name=current_cohort,
        defaults={'assign_datetime': get_utcnow(), 'enrollment_cohort': False})

    # Take child offschedule
    onschedule_obj = get_onschedule_obj(subject_identifier, enrol_cohort)
    take_off_schedule(onschedule_obj)
    
    # Put child onschedule
    schedule_type = get_schedule_type(onschedule_obj)
    put_onschedule(current_cohort, schedule_type, child_schedule_dict,
                   onschedule_obj.onschedule_datetime,
                   subject_identifier, ChildAppointment, )

    # Take caregiver offschedule
    onschedule_obj = get_onschedule_obj(caregiver_subject_identifier, enrol_cohort)
    take_off_schedule(onschedule_obj)

    # Put caregiver onschedule
    schedule_type = get_schedule_type(onschedule_obj)
    child_count = cohort_change.get('child_count')
    onschedule_model, schedule_name = put_onschedule(
        current_cohort, schedule_type, caregiver_schedule_dict,
        onschedule_obj.onschedule_datetime,
        caregiver_subject_identifier, Appointment, is_caregiver=True, child_count=str(child_count))

    # Update child subject identifier
    update_onschedule_model(caregiver_subject_identifier, subject_identifier,
                            onschedule_model, schedule_name, )
    print('done', subject_identifier)

  0%|          | 0/291 [00:00<?, ?it/s]

start B142-040990078-3-10


  0%|          | 1/291 [00:01<05:58,  1.24s/it]

done B142-040990078-3-10
start B142-040990708-5-10


  1%|          | 2/291 [00:02<05:09,  1.07s/it]

...

done B142-040990488-4-10
start B142-040990549-3-10


  2%|▏         | 6/291 [00:06<04:53,  1.03s/it]

done B142-040990549-3-10
start B142-040990795-2-10


  2%|▏         | 6/291 [00:06<05:24,  1.14s/it]


KeyError: 'None'